**Primeiro Trabalho da Disciplina	Processamento de Dados em Larga Escala**

**Discente: Valéria Cristina A. R. de Figueredo - Individual**

**Docente: Anderson Neves**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade "dask[dataframe]"

In [ ]:
!pip install --upgrade "dask[distributed]"

In [ ]:
!pip install --upgrade pyarrow

## Sobre os dados

O arquivo CSV contém eventos 'click' ou 'view' no tempo, de usuários em anúncios de determinadas campanhas.

**Descrição das colunas:**  
timestamp,user_id,action,adId,campaignId

**Amostra:**  
2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01  
2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02  
2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02  
2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03

**Nome do arquivo CSV:**  
data/ad_action.csv

## Sobre as questões

As questões devem ser respondidas usando a tecnologia Dask.

Quando utilizar pandas tenha cuidado para evitar estouro de memória, sempre imaginado que vai executar o código com uma grande massa de dados.

Mesmo que não consiga terminar alguma questão, favor enviar, porque parte do código pode valer alguma pontuação.

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()
client = Client(cluster)
client

/usr/local/lib/python3.11/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33925 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:36835
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:33925/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:40753'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46763'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:38089 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:38089
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:36388
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:40419 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:40419
IN

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:33925/status,
Dashboard: http://127.0.0.1:33925/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36835,Workers: 2
Dashboard: http://127.0.0.1:33925/status,Total threads: 2
Started: Just now,Total memory: 12.67 GiB
Comm: tcp://127.0.0.1:38089,Total threads: 1
Dashboard: http://127.0.0.1:44753/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:40753,


In [ ]:
df = dd.read_csv('drive/MyDrive/data/ad_action.csv', header=None, parse_dates=[0])
df.columns = ['timestamp','user_id','action','adId','campaignId']
df

,timestamp,user_id,action,adId,campaignId
npartitions=1,,,,,
,datetime64[ns],string,string,string,string
,...,...,...,...,...


In [ ]:
df.head(5)

,timestamp,user_id,action,adId,campaignId
0,2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01
1,2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02
2,2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02
3,2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03
4,2016-01-02 04:57:00,fef9a98c-d73e-48ef-b2cb-766ba85dc3e3,view,adId_02,campaignId_02


## 1) Quais são as top 3 campanhas que geraram mais eventos? Ordene pela quantidade de eventos (2,5 pontos)

In [ ]:
top_popular_campaign = df.groupby('campaignId')\
    ['timestamp']\
    .count()\
    .rename('event_count')\
    .nlargest(3)\
    .reset_index()
top_popular_campaign.compute()


,campaignId,event_count
0,campaignId_02,91216
1,campaignId_03,87036
2,campaignId_01,76461


As top 3 campanhas que geraram mais eventos (click e view) foram respectivamente campaignId_02, campaignId_03 e campaignId_01.Elas estão ordenadas pela quantidade de eventos.

## 2) Qual campanha teve mais clicks? (2,5 pontos)

In [ ]:
top_click_campaign = df[df['action'] == 'click'] \
    .groupby('campaignId') \
    ['timestamp']\
    .count()\
    .rename('event_count')\
    .nlargest(1)\
    .reset_index()
top_click_campaign.compute()

,campaignId,event_count
0,campaignId_02,63983


A campanha que teve mais clicks foi a campaignId_02, com 63.983 clicks.

## 3) Qual mês teve o maior total de eventos acumulado? (2,5 pontos)

In [ ]:
df.head()

,timestamp,user_id,action,adId,campaignId
0,2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01
1,2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02
2,2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02
3,2016-03-26 06:27:00,78aa2467-b502-413b-94e9-04ec8210bd13,click,adId_07,campaignId_03
4,2016-01-02 04:57:00,fef9a98c-d73e-48ef-b2cb-766ba85dc3e3,view,adId_02,campaignId_02


In [ ]:
df['month'] = df['timestamp'].dt.month
df.head(3)

,timestamp,user_id,action,adId,campaignId,month
0,2016-09-21 22:11:00,7c74953c-66cc-48bd-9d02-a02bf039cf3f,click,adId_09,campaignId_01,9
1,2016-06-25 18:29:00,676a083e-2f8e-4ff2-9ec2-270f7f9d6033,view,adId_09,campaignId_02,6
2,2016-02-14 19:03:00,77158997-0dfa-48b7-9149-973dc151ef8d,click,adId_02,campaignId_02,2


In [ ]:
month_events_acumulate = df.groupby('month')\
    ['timestamp']\
    .count()\
    .rename('event_count')\
    .nlargest(12)\
    .reset_index()
month_events_acumulate.compute()


,month,event_count
0,1,25800
1,3,21377
2,10,21363
3,8,21362
4,5,21224
5,7,21183
6,6,20657
7,11,20646
8,9,20627
9,4,20558


O mês com o maior total de eventos acumulados foi janeiro, com o total de 25.800.

## 4) Nas situações onde existe um evento de view seguido de um evento de click criados pelo mesmo usuário no mesmo anúncio e campanha, quais são os 5 pares de anúncio e campanha com menores médias de tempo entre os dois eventos (2,5 pontos)

In [ ]:

# Filtro nos eventos de "view" e "click"
views = df[df['action'] == 'view']
clicks = df[df['action'] == 'click']

# Realização do merge dos eventos de "view" e "click" com base no mesmo usuário, anúncio e campanha
merged = dd.merge(views, clicks, on=['user_id', 'adId', 'campaignId'], suffixes=('_view', '_click'))

# Cálculo da diferença de tempo entre os eventos "view" e "click"
merged['time_diff'] = (merged['timestamp_click'] - merged['timestamp_view']).dt.total_seconds()

# Agrupamento por 'adId' e 'campaignId' e cálculo da média do tempo de diferença
avg_time_diff = merged.groupby(['adId', 'campaignId'])['time_diff'].mean().compute()

# Ordenação dos pares por tempo médio e selecionando os 5 menores
top_5 = avg_time_diff.nsmallest(5)

# Exibição dos resultados
print("Os 5 pares de anúncio e campanha com menores médias de tempo entre os eventos são:")
print(top_5)

Os 5 pares de anúncio e campanha com menores médias de tempo entre os eventos são:
adId     campaignId   
adId_09  campaignId_01   -441369.191503
adId_02  campaignId_03   -366346.574299
adId_07  campaignId_01   -301518.857697
adId_05  campaignId_02   -233095.751671
adId_04  campaignId_01   -193454.256990
Name: time_diff, dtype: float64
